In [1]:
"""
Enhanced Robust GP Optimizer with Anti-Stagnation + Local Penalization + Diversity
"""

import numpy as np
import pickle
import os
import warnings
from scipy.stats import qmc, norm
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, RBF, ConstantKernel, WhiteKernel
from qg_model import QGTwoLayerModel
from scipy.ndimage import uniform_filter

warnings.filterwarnings('ignore', category=UserWarning, module='sklearn.gaussian_process')
warnings.filterwarnings('ignore', message='The optimal value found for dimension')

class Colors:
    """Compact color printing"""
    @staticmethod
    def green(t): return f"\033[92m{t}\033[0m"
    @staticmethod
    def cyan(t): return f"\033[96m{t}\033[0m"
    @staticmethod
    def yellow(t): return f"\033[93m{t}\033[0m"
    @staticmethod
    def red(t): return f"\033[91m{t}\033[0m"
    @staticmethod
    def bold(t): return f"\033[1m{t}\033[0m"
    @staticmethod
    def star(t): return f"\033[93m\033[1m★ {t}\033[0m"

# Parameter configuration
PARAM_BOUNDS = {
    'viscosity_scale': {'bounds': (0.5, 5.0), 'type': 'linear'},
    'drag_scale': {'bounds': (0.5, 3.0), 'type': 'linear'},
    'eddy_diffusivity': {'bounds': (1e3, 1e5), 'type': 'log'},
    'smagorinsky_coeff': {'bounds': (0.0, 0.3), 'type': 'linear'},
    'energy_correction': {'bounds': (-0.01, 0.01), 'type': 'linear'},
    'enstrophy_correction': {'bounds': (0.0, 1e-6), 'type': 'log'},
}
PARAM_NAMES = list(PARAM_BOUNDS.keys())
N_PARAMS = len(PARAM_NAMES)

# Input warping for log-scale parameters
def warp_parameters(params_array):
    """Transform to warped space: log params → log space, linear → [0,1]"""
    warped = np.zeros(N_PARAMS)
    for i, name in enumerate(PARAM_NAMES):
        val, info = params_array[i], PARAM_BOUNDS[name]
        lower, upper = info['bounds']
        if info['type'] == 'log':
            log_lower, log_upper = np.log10(lower) if lower > 0 else -10, np.log10(upper)
            warped[i] = (np.log10(val + 1e-20) - log_lower) / (log_upper - log_lower)
        else:
            warped[i] = (val - lower) / (upper - lower)
    return warped

def unwarp_parameters(warped_array):
    """Transform from warped space back to original space"""
    params = np.zeros(N_PARAMS)
    for i, name in enumerate(PARAM_NAMES):
        info = PARAM_BOUNDS[name]
        lower, upper = info['bounds']
        if info['type'] == 'log':
            log_lower, log_upper = np.log10(lower) if lower > 0 else -10, np.log10(upper)
            params[i] = 10 ** (warped_array[i] * (log_upper - log_lower) + log_lower)
        else:
            params[i] = warped_array[i] * (upper - lower) + lower
        params[i] = np.clip(params[i], lower, upper)
    return params

# ============================================================================
# SMART INITIALIZATION
# ============================================================================

def generate_smart_initial_samples(n_samples):
    """Combine Latin Hypercube and Sobol for better space-filling"""
    n_lhs = n_samples // 2
    n_sobol = n_samples - n_lhs
    
    lhs_sampler = qmc.LatinHypercube(d=N_PARAMS, seed=42)
    lhs_samples = lhs_sampler.random(n=n_lhs)
    
    sobol_sampler = qmc.Sobol(d=N_PARAMS, seed=42)
    sobol_samples = sobol_sampler.random(n=n_sobol)
    
    unit_samples = np.vstack([lhs_samples, sobol_samples])
    
    samples = []
    for unit_sample in unit_samples:
        samples.append(unwarp_parameters(unit_sample))
    
    return np.array(samples)

# ============================================================================
# ENSEMBLE GP
# ============================================================================

class EnsembleGP:
    """Ensemble of Gaussian Processes with different kernels"""
    
    def __init__(self, n_models=5):
        self.models = []
        
        kernels = [
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            Matern(length_scale=[1.0]*N_PARAMS, length_scale_bounds=(1e-3, 1e3), nu=1.5) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
            
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            Matern(length_scale=[1.0]*N_PARAMS, length_scale_bounds=(1e-3, 1e3), nu=2.5) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
            
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            RBF(length_scale=[1.0]*N_PARAMS, length_scale_bounds=(1e-3, 1e3)) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
            
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            Matern(length_scale=[0.5]*N_PARAMS, length_scale_bounds=(1e-3, 1e2), nu=2.5) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
            
            ConstantKernel(1.0, (1e-3, 1e3)) * 
            RBF(length_scale=[2.0]*N_PARAMS, length_scale_bounds=(1e-2, 1e3)) +
            WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-1)),
        ]
        
        for kernel in kernels[:n_models]:
            self.models.append(GaussianProcessRegressor(
                kernel=kernel,
                alpha=1e-6,
                normalize_y=True,
                n_restarts_optimizer=15,
                random_state=None
            ))
    
    def fit(self, X, y):
        """Fit all models"""
        for i, model in enumerate(self.models):
            try:
                model.fit(X, y)
            except Exception as e:
                print(f"  Warning: Model {i} fitting failed: {e}")
    
    def predict(self, X, return_std=True):
        """Ensemble prediction"""
        X = np.atleast_2d(X)
        
        if return_std:
            predictions = []
            uncertainties = []
            
            for model in self.models:
                try:
                    mu, sigma = model.predict(X, return_std=True)
                    predictions.append(mu)
                    uncertainties.append(sigma)
                except:
                    continue
            
            if len(predictions) == 0:
                return np.zeros(len(X)), np.ones(len(X))
            
            mean = np.mean(predictions, axis=0)
            std = np.max(uncertainties, axis=0)
            
            return mean, std
        else:
            predictions = []
            for model in self.models:
                try:
                    predictions.append(model.predict(X))
                except:
                    continue
            
            return np.mean(predictions, axis=0) if predictions else np.zeros(len(X))

# Trust region with reset capability
class TrustRegion:
    def __init__(self):
        self.trust_radius, self.best_center = 0.5, None
        self.success_count, self.fail_count = 0, 0
        self.min_radius, self.max_radius = 0.05, 1.0
    
    def get_trust_region_bounds(self):
        if self.best_center is None:
            return [(0, 1)] * N_PARAMS
        bounds = []
        for i in range(N_PARAMS):
            center, half_width = self.best_center[i], self.trust_radius / 2
            bounds.append((max(0.0, center - half_width), min(1.0, center + half_width)))
        return bounds
    
    def update(self, new_best_found, new_center=None):
        if new_best_found:
            self.success_count += 1
            self.fail_count = 0
            if new_center is not None:
                self.best_center = new_center
            if self.success_count >= 3:
                self.trust_radius = min(self.max_radius, self.trust_radius * 1.5)
                self.success_count = 0
                print(f"  → Trust region expanded to {self.trust_radius:.2f}")
        else:
            self.fail_count += 1
            self.success_count = 0
            if self.fail_count >= 3:
                self.trust_radius = max(self.min_radius, self.trust_radius * 0.5)
                self.fail_count = 0
                print(f"  → Trust region shrunk to {self.trust_radius:.2f}")
    
    def reset_for_exploration(self):
        self.trust_radius = 0.8
        self.success_count, self.fail_count = 0, 0
        print(f"  → Trust region RESET to {self.trust_radius:.2f}")

# NEW: Hybrid acquisition with LOCAL PENALIZATION (prevents clustering)
def hybrid_acquisition_with_penalization(X, gp, best_y, X_samples, xi=0.01, kappa=2.0, weight_ei=0.6, penalization_weight=0.3):
    """Hybrid EI+UCB with local penalization to avoid sampling near existing points"""
    X = np.atleast_2d(X)
    mu, sigma = gp.predict(X, return_std=True)
    
    # Expected Improvement
    with np.errstate(divide='warn', invalid='warn'):
        imp = best_y - mu - xi
        Z = imp / (sigma + 1e-9)
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    
    # Upper Confidence Bound
    ucb = -(mu - kappa * sigma)
    
    # Normalize
    ei_norm = (ei - ei.min()) / (ei.max() - ei.min() + 1e-9)
    ucb_norm = (ucb - ucb.min()) / (ucb.max() - ucb.min() + 1e-9)
    
    # Base acquisition
    acq = weight_ei * ei_norm + (1 - weight_ei) * ucb_norm
    
    # NEW: Local penalization - reduce acquisition near existing samples
    if len(X_samples) > 0 and penalization_weight > 0:
        X_samples_warped = np.array([warp_parameters(x) for x in X_samples])
        min_distances = np.min([np.linalg.norm(X - x_sample, axis=1) for x_sample in X_samples_warped], axis=0)
        # Penalize points close to existing samples (distance < 0.1 in warped space)
        penalty = np.exp(-10 * min_distances)  # High penalty when distance < 0.1
        acq = acq * (1 - penalization_weight * penalty)
    
    return acq

# Acquisition optimizer with multi-start
def optimize_acquisition_multistart(acquisition_fn, bounds, n_starts=20, n_random=500):
    """Multi-start optimization of acquisition function"""
    best_acq, best_x = -np.inf, None
    
    # Random sampling
    random_samples = np.random.uniform([b[0] for b in bounds], [b[1] for b in bounds], size=(n_random, N_PARAMS))
    acq_random = acquisition_fn(random_samples)
    best_random_idx = np.argmax(acq_random)
    if acq_random[best_random_idx] > best_acq:
        best_acq, best_x = acq_random[best_random_idx], random_samples[best_random_idx]
    
    # Gradient-based optimization
    for _ in range(n_starts):
        x0 = np.array([np.random.uniform(b[0], b[1]) for b in bounds])
        result = minimize(lambda x: -acquisition_fn(x.reshape(1, -1))[0], x0, method='L-BFGS-B', bounds=bounds)
        if result.success and -result.fun > best_acq:
            best_acq, best_x = -result.fun, result.x
    
    return best_x

# Simulation runner (compact)
def run_lowres_with_params(params_array, config_base, highres_results, sim_days=180):
    from main_comparison import run_simulation
    config = config_base.copy()
    config['subgrid_params'] = {PARAM_NAMES[i]: float(params_array[i]) for i in range(N_PARAMS)}
    
    print(f"\n{'='*70}\nTesting parameters:")
    for param_name, val in config['subgrid_params'].items():
        print(f"  {param_name}: {val:.6e}")
    
    try:
        results = run_simulation(config, sim_days=sim_days, save_interval_hours=12)
        loss, detailed = compute_loss(results, highres_results, return_fields=True)
        if not np.isfinite(loss):
            print(Colors.yellow(f"  ⚠ Loss not finite: {loss}"))
            return np.nan, None, None
        print(f"  Loss: {Colors.green(f'{loss:.6f}')}")
        return loss, results, detailed
    except Exception as e:
        print(Colors.yellow(f"  ⚠ Simulation failed: {e}"))
        return np.nan, None, None

# Loss computation (compact)
def compute_loss(lowres_results, highres_results, n_days_avg=30, return_fields=False):
    nx_hr, ny_hr = highres_results['config']['nx'], highres_results['config']['ny']
    nx_lr, ny_lr = lowres_results['config']['nx'], lowres_results['config']['ny']
    coarsen_factor_x, coarsen_factor_y = nx_hr // nx_lr, ny_hr // ny_lr
    
    times_hr, times_lr = highres_results['times'], lowres_results['times']
    indices_hr = np.where(times_hr >= times_hr[-1] - n_days_avg)[0]
    indices_lr = np.where(times_lr >= times_lr[-1] - n_days_avg)[0]
    
    q1_hr_avg = np.mean([highres_results['q1_history'][i] for i in indices_hr], axis=0)
    q2_hr_avg = np.mean([highres_results['q2_history'][i] for i in indices_hr], axis=0)
    q1_lr_avg = np.mean([lowres_results['q1_history'][i] for i in indices_lr], axis=0)
    q2_lr_avg = np.mean([lowres_results['q2_history'][i] for i in indices_lr], axis=0)
    
    model_hr, model_lr = highres_results['model'], lowres_results['model']
    psi1_hr_avg, psi2_hr_avg = model_hr.q_to_psi(q1_hr_avg, q2_hr_avg)
    psi1_lr_avg, psi2_lr_avg = model_lr.q_to_psi(q1_lr_avg, q2_lr_avg)
    
    H1, H2, H_total = model_hr.H1, model_hr.H2, model_hr.H1 + model_hr.H2
    q_bt_hr = (H1 * q1_hr_avg + H2 * q2_hr_avg) / H_total
    psi_bt_hr = (H1 * psi1_hr_avg + H2 * psi2_hr_avg) / H_total
    q_bt_lr = (H1 * q1_lr_avg + H2 * q2_lr_avg) / H_total
    psi_bt_lr = (H1 * psi1_lr_avg + H2 * psi2_lr_avg) / H_total
    
    def coarsen(field, fx, fy):
        return uniform_filter(field, size=(fy, fx), mode='wrap')[::fy, ::fx]
    
    q_bt_hr_coarse = coarsen(q_bt_hr, coarsen_factor_x, coarsen_factor_y)
    psi_bt_hr_coarse = coarsen(psi_bt_hr, coarsen_factor_x, coarsen_factor_y)
    
    nrmse = lambda pred, target: np.sqrt(np.mean((pred - target)**2)) / (np.std(target) + 1e-20)
    loss_q_bt, loss_psi_bt = nrmse(q_bt_lr, q_bt_hr_coarse), nrmse(psi_bt_lr, psi_bt_hr_coarse)
    weight_pv, weight_psi = 0.6, 0.4
    total_loss = weight_pv * loss_q_bt + weight_psi * loss_psi_bt
    
    if return_fields:
        return total_loss, {'q_bt_hr_coarse': q_bt_hr_coarse, 'psi_bt_hr_coarse': psi_bt_hr_coarse,
                           'q_bt_lr': q_bt_lr, 'psi_bt_lr': psi_bt_lr, 'loss_q_bt': loss_q_bt,
                           'loss_psi_bt': loss_psi_bt, 'total_loss': total_loss}
    return total_loss

# Enhanced GP Optimizer with smart robustness features
class EnhancedGPOptimizer:
    """
    Enhanced GP with 3 smart robustness features:
    1. Anti-stagnation (auto-restart after 15 iterations WITHOUT improvement)
    2. Local penalization (prevents clustering near existing samples)
    3. 8-model ensemble (robust predictions from diverse kernels)
    """
    def __init__(self, n_initial_samples=12):
        self.n_initial_samples = n_initial_samples
        self.X_samples, self.y_samples, self.detailed_outputs = [], [], []
        self.best_loss, self.best_params, self.best_iteration = np.inf, None, -1
        self.iteration, self.iterations_without_improvement = 0, 0
        self.stagnation_threshold = 15
        self.gp = EnsembleGP(n_models=8)  # 8 models for robustness
        self.trust_region = TrustRegion()
    
    def optimize(self, config_base, highres_results, max_iterations=100):
        print("\n" + "="*70)
        print("ENHANCED GP: 8-MODEL ENSEMBLE + LOCAL PENALIZATION + ANTI-STAGNATION")
        print("="*70)
        print("Features:")
        print("  ✓ 8-model ensemble (diverse kernels for robust predictions)")
        print("  ✓ Local penalization (avoids clustering, ensures space coverage)")
        print(Colors.bold("  ✓ Anti-stagnation (auto-restart ONLY when stuck >15 iterations)"))
        print(f"  Max iterations: {max_iterations}")
        print("="*70)
        
        # Phase 1: Initial sampling
        n_existing = len(self.X_samples)
        if n_existing < self.n_initial_samples:
            print(f"\n{'='*70}\nPHASE 1: SMART INITIALIZATION\n{'='*70}")
            initial_samples = generate_smart_initial_samples(self.n_initial_samples)
            
            for i, params in enumerate(initial_samples[n_existing:]):
                print(f"\n[Initial {i+1+n_existing}/{self.n_initial_samples}]")
                loss, results, detailed = run_lowres_with_params(params, config_base, highres_results)
                self.X_samples.append(params)
                self.y_samples.append(loss)
                self.detailed_outputs.append(detailed)
                
                if np.isfinite(loss) and loss < self.best_loss:
                    self.best_loss, self.best_params = loss, params.copy()
                    self.best_iteration, self.iterations_without_improvement = len(self.X_samples) - 1, 0
                    print(Colors.star(f"NEW BEST: {Colors.green(f'{loss:.6f}')}"))
                self.save_progress()
        
        # Phase 2: Bayesian optimization with smart features
        print(f"\n{'='*70}\nPHASE 2: BAYESIAN OPTIMIZATION\n{'='*70}")
        
        for iteration in range(len(self.X_samples), max_iterations):
            self.iteration, self.iterations_without_improvement = iteration, self.iterations_without_improvement + 1
            
            print(f"\n{'='*70}\n{Colors.cyan(f'ITERATION {iteration + 1}/{max_iterations}')}\n{'='*70}")
            
            # Stagnation check (ONLY trigger when stuck)
            if self.iterations_without_improvement >= self.stagnation_threshold:
                print(Colors.red(f"\n⚠ STAGNATION: {self.iterations_without_improvement} iterations w/o improvement"))
                print(Colors.yellow("→ Triggering exploration restart"))
                self.trigger_exploration_restart()
            
            # Fit GP
            X_warped = np.array([warp_parameters(x) for x in self.X_samples])
            y_array = np.array(self.y_samples)
            valid_mask = np.isfinite(y_array)
            n_valid = np.sum(valid_mask)
            
            print(f"Valid samples: {Colors.cyan(str(n_valid))}/{len(y_array)}")
            
            kappa = self.get_adaptive_kappa()
            if kappa > 2.0:
                print(Colors.yellow(f"  ℹ Increased exploration: kappa = {kappa:.1f}"))
            
            if n_valid < 5:
                print(Colors.yellow("  ⚠ Too few valid samples, random exploration"))
                next_params = unwarp_parameters(np.random.uniform(0, 1, N_PARAMS))
            else:
                X_valid, y_valid = X_warped[valid_mask], y_array[valid_mask]
                print("  Fitting 8-model ensemble GP...")
                self.gp.fit(X_valid, y_valid)
                
                # Print parameter importance
                try:
                    kernel = self.gp.models[0].kernel_
                    if hasattr(kernel, 'k2'):
                        length_scales = kernel.k2.length_scale
                        if hasattr(length_scales, '__len__'):
                            print("  Parameter importance (smaller = more important):")
                            for i, name in enumerate(PARAM_NAMES):
                                imp = "HIGH" if length_scales[i] < 1.0 else "med" if length_scales[i] < 10 else "low"
                                print(f"    {name}: {Colors.cyan(f'{length_scales[i]:.2f}')} ({imp})")
                except:
                    pass
                
                # Update trust region
                if self.best_params is not None:
                    self.trust_region.best_center = warp_parameters(self.best_params)
                
                tr_bounds = self.trust_region.get_trust_region_bounds()
                print(f"  Trust region: {Colors.cyan(f'{self.trust_region.trust_radius:.2f}')}")
                
                # Optimize with LOCAL PENALIZATION
                print(f"  Optimizing acquisition (kappa={kappa:.1f}, w/ local penalization)...")
                best_y = np.min(y_valid)
                acq_fn = lambda X: hybrid_acquisition_with_penalization(
                    X, self.gp, best_y, self.X_samples, xi=0.01, kappa=kappa, penalization_weight=0.3
                )
                
                next_params_warped = optimize_acquisition_multistart(acq_fn, tr_bounds, n_starts=20, n_random=1000)
                acq_val = acq_fn(next_params_warped.reshape(1, -1))[0]
                print(f"  Selected point (acq={Colors.cyan(f'{acq_val:.4f}')})")
                next_params = unwarp_parameters(next_params_warped)
            
            # Evaluate
            loss, results, detailed = run_lowres_with_params(next_params, config_base, highres_results)
            self.X_samples.append(next_params)
            self.y_samples.append(loss)
            self.detailed_outputs.append(detailed)
            
            # Update best
            new_best = False
            if np.isfinite(loss) and loss < self.best_loss:
                self.best_loss, self.best_params = loss, next_params.copy()
                self.best_iteration, self.iterations_without_improvement = iteration, 0
                new_best = True
                print(Colors.star(f"NEW BEST: {Colors.green(f'{loss:.6f}')}"))
            
            self.trust_region.update(new_best, warp_parameters(self.best_params) if new_best else None)
            self.print_status()
            self.save_progress()
        
        return self.get_best_params()
    
    def get_adaptive_kappa(self):
        """Adaptive kappa: higher when stuck"""
        if self.iterations_without_improvement < 6:
            return 2.0
        elif self.iterations_without_improvement < 10:
            return 3.0
        return 4.0
    
    def trigger_exploration_restart(self):
        """Reset trust region and add random sample"""
        self.trust_region.reset_for_exploration()
        print(Colors.yellow("  → Random sample will be added next"))
        self.iterations_without_improvement = 0
        print(Colors.green("  ✓ Restart complete"))
    
    def print_status(self):
        """Print status with stagnation warning"""
        n_valid = np.sum(np.isfinite(self.y_samples))
        n_failed = len(self.y_samples) - n_valid
        print(f"\n{Colors.bold('Status:')}")
        print(f"  Valid: {Colors.cyan(str(n_valid))}/{len(self.y_samples)}")
        print(f"  Failed: {Colors.yellow(str(n_failed))}")
        print(f"  {Colors.bold('Best loss:')} {Colors.green(f'{self.best_loss:.6f}')} {Colors.cyan(f'(iteration {self.best_iteration + 1})')}")
        stag_str = f"{self.iterations_without_improvement}/{self.stagnation_threshold}"
        stag_str = Colors.yellow(stag_str) if self.iterations_without_improvement >= 10 else Colors.cyan(stag_str)
        print(f"  Iterations w/o improvement: {stag_str}")
    
    def get_best_params(self):
        if self.best_params is None:
            raise ValueError("No valid parameters found!")
        return {PARAM_NAMES[i]: float(self.best_params[i]) for i in range(N_PARAMS)}
    
    def save_progress(self, filename='enhanced_gp_progress.pkl'):
        data = {
            'X_samples': self.X_samples, 'y_samples': self.y_samples, 'detailed_outputs': self.detailed_outputs,
            'best_loss': self.best_loss, 'best_params': self.best_params, 'best_iteration': self.best_iteration,
            'iteration': self.iteration, 'iterations_without_improvement': self.iterations_without_improvement,
            'trust_region_state': {'radius': self.trust_region.trust_radius, 'center': self.trust_region.best_center,
                                  'success_count': self.trust_region.success_count, 'fail_count': self.trust_region.fail_count}
        }
        with open(filename, 'wb') as f:
            pickle.dump(data, f)
        print(f"  ✓ Progress saved")
    
    @classmethod
    def load_progress(cls, filename='enhanced_gp_progress.pkl'):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        
        optimizer = cls()
        optimizer.X_samples, optimizer.y_samples = data['X_samples'], data['y_samples']
        optimizer.detailed_outputs = data['detailed_outputs']
        optimizer.best_loss, optimizer.best_params = data['best_loss'], data['best_params']
        optimizer.best_iteration, optimizer.iteration = data['best_iteration'], data['iteration']
        optimizer.iterations_without_improvement = data.get('iterations_without_improvement', 0)
        
        if 'trust_region_state' in data:
            tr = data['trust_region_state']
            optimizer.trust_region.trust_radius, optimizer.trust_region.best_center = tr['radius'], tr['center']
            optimizer.trust_region.success_count, optimizer.trust_region.fail_count = tr['success_count'], tr['fail_count']
        
        print(f"✓ Loaded checkpoint:")
        print(f"  Iterations: {len(optimizer.X_samples)}")
        n_valid = np.sum(np.isfinite(optimizer.y_samples))
        print(f"  Valid: {Colors.cyan(str(n_valid))}/{len(optimizer.y_samples)}")
        print(f"  {Colors.bold('Best loss:')} {Colors.green(f'{optimizer.best_loss:.6f}')}")
        stag = optimizer.iterations_without_improvement
        print(f"  Iterations w/o improvement: {Colors.yellow(str(stag))}/{optimizer.stagnation_threshold}")
        if stag >= optimizer.stagnation_threshold:
            print(Colors.red("  ⚠ Will trigger restart on next iteration!"))
        
        return optimizer

# Main function (compact)
def main(checkpoint_file='enhanced_gp_progress.pkl', max_iterations=100):
    """Main optimization routine"""
    if not os.path.exists('highres_results.pkl'):
        print("\n✗ Error: highres_results.pkl not found!")
        return
    
    with open('highres_results.pkl', 'rb') as f:
        highres_results = pickle.load(f)
    print(f"\n✓ Loaded high-res: {highres_results['config']['nx']}x{highres_results['config']['ny']}")
    
    from main_comparison import config_lowres
    config_base = config_lowres.copy()
    
    if os.path.exists(checkpoint_file):
        print(f"\n✓ Checkpoint found")
        optimizer = EnhancedGPOptimizer.load_progress(checkpoint_file)
    else:
        print(f"\n✓ Starting new optimization")
        optimizer = EnhancedGPOptimizer(n_initial_samples=12)
    
    best_params = optimizer.optimize(config_base, highres_results, max_iterations)
    
    print("\n" + "="*70)
    print("OPTIMIZATION COMPLETE")
    print("="*70)
    
    n_valid = np.sum(np.isfinite(optimizer.y_samples))
    n_failed = len(optimizer.y_samples) - n_valid
    print(f"\nTotal iterations: {len(optimizer.y_samples)}")
    print(f"  Valid: {Colors.cyan(str(n_valid))}")
    print(f"  Failed: {Colors.yellow(str(n_failed))}")
    print(f"\n{Colors.bold('Best loss:')} {Colors.green(f'{optimizer.best_loss:.6f}')}")
    print(f"{Colors.bold('Best parameters:')}")
    for name, val in best_params.items():
        print(f"  {name}: {Colors.cyan(f'{val:.6e}')}")
    
    with open('enhanced_gp_optimal_params.pkl', 'wb') as f:
        pickle.dump(best_params, f)
    with open('enhanced_gp_optimal_config.txt', 'w') as f:
        f.write("'subgrid_params': {\n")
        for name, val in best_params.items():
            f.write(f"    '{name}': {val:.6e},\n")
        f.write("}\n")
    
    print("\n✓ Saved: enhanced_gp_optimal_params.pkl")
    print("✓ Saved: enhanced_gp_optimal_config.txt")
    return optimizer, best_params

if __name__ == "__main__":
    optimizer, best_params = main(max_iterations=100)


✓ Loaded high-res: 512x256


/panfs/ccds02/nobackup/people/afahad/pkgs/qg/lib/python3.10/site-packages/scipy/stats/_qmc.py:993: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)



✓ Starting new optimization

ENHANCED GP: 8-MODEL ENSEMBLE + LOCAL PENALIZATION + ANTI-STAGNATION
Features:
  ✓ 8-model ensemble (diverse kernels for robust predictions)
  ✓ Local penalization (avoids clustering, ensures space coverage)
  ✓ Anti-stagnation (auto-restart ONLY when stuck >15 iterations)
  Max iterations: 100

PHASE 1: SMART INITIALIZATION

[Initial 1/12]

Testing parameters:
  viscosity_scale: 2.919533e+00
  drag_scale: 1.983801e+00
  eddy_diffusivity: 2.401416e+03
  smagorinsky_coeff: 2.151316e-01
  energy_correction: -6.980591e-03
  enstrophy_correction: 4.818573e-08

Running LowRes_64x32 Simulation
Grid: 64 x 32
Resolution: 31.2 km per grid point

Subgrid Parameters:
  viscosity_scale: 2.9195329635830274
  drag_scale: 1.9838006501033116
  eddy_diffusivity: 2401.4157795559904
  smagorinsky_coeff: 0.2151315985470318
  energy_correction: -0.006980591159625498
  enstrophy_correction: 4.818572882784813e-08

Initial Energy: 5.940e+02
Initial Enstrophy: 8.404e-12

Integrati

100%|██████████| 8640/8640 [00:26<00:00, 331.24it/s]



LowRes_64x32 Simulation Complete!
  Loss: 1.472544
★ NEW BEST: 1.472544
  ✓ Progress saved

[Initial 2/12]

Testing parameters:
  viscosity_scale: 2.179145e+00
  drag_scale: 5.891399e-01
  eddy_diffusivity: 1.952670e+04
  smagorinsky_coeff: 1.274807e-01
  energy_correction: 8.764007e-03
  enstrophy_correction: 1.118983e-08

Running LowRes_64x32 Simulation
Grid: 64 x 32
Resolution: 31.2 km per grid point

Subgrid Parameters:
  viscosity_scale: 2.179145223507235
  drag_scale: 0.5891398728012692
  eddy_diffusivity: 19526.697082557086
  smagorinsky_coeff: 0.12748070310522164
  energy_correction: 0.008764006585891397
  enstrophy_correction: 1.1189826190569524e-08

Initial Energy: 5.940e+02
Initial Enstrophy: 8.404e-12

Integrating...


...


LowRes_64x32 Simulation Complete!
  Loss: 0.265953

Status:
  Valid: 98/98
  Failed: 0
  Best loss: 0.127704 (iteration 59)
  Iterations w/o improvement: 9/15
  ✓ Progress saved

ITERATION 99/100
Valid samples: 98/98
  ℹ Increased exploration: kappa = 4.0
  Fitting 8-model ensemble GP...
  Trust region: 0.10
  Optimizing acquisition (kappa=4.0, w/ local penalization)...
  Selected point (acq=0.0000)

Testing parameters:
  viscosity_scale: 2.818434e+00
  drag_scale: 1.046383e+00
  eddy_diffusivity: 4.054854e+04
  smagorinsky_coeff: 1.696873e-01
  energy_correction: 5.362553e-04
  enstrophy_correction: 6.536711e-09

Running LowRes_64x32 Simulation
Grid: 64 x 32
Resolution: 31.2 km per grid point

Subgrid Parameters:
  viscosity_scale: 2.8184339884340326
  drag_scale: 1.0463825428706541
  eddy_diffusivity: 40548.54202624742
  smagorinsky_coeff: 0.16968728155625198
  energy_correction: 0.000536255321147254
  enstrophy_correction: 6.536711256705641e-09

Initial Energy: 5.940e+02
Initial En

100%|██████████| 8640/8640 [00:26<00:00, 327.48it/s]



LowRes_64x32 Simulation Complete!
  Loss: 0.228346

Status:
  Valid: 99/99
  Failed: 0
  Best loss: 0.127704 (iteration 59)
  Iterations w/o improvement: 10/15
  ✓ Progress saved

ITERATION 100/100
Valid samples: 99/99
  ℹ Increased exploration: kappa = 4.0
  Fitting 8-model ensemble GP...
  Trust region: 0.10
  Optimizing acquisition (kappa=4.0, w/ local penalization)...
  Selected point (acq=0.0000)

Testing parameters:
  viscosity_scale: 2.555523e+00
  drag_scale: 9.942436e-01
  eddy_diffusivity: 5.814279e+04
  smagorinsky_coeff: 1.576287e-01
  energy_correction: -2.412856e-04
  enstrophy_correction: 8.910816e-09

Running LowRes_64x32 Simulation
Grid: 64 x 32
Resolution: 31.2 km per grid point

Subgrid Parameters:
  viscosity_scale: 2.555522755413018
  drag_scale: 0.9942435714862592
  eddy_diffusivity: 58142.7911235618
  smagorinsky_coeff: 0.15762866666292022
  energy_correction: -0.00024128555915123133
  enstrophy_correction: 8.910816048614196e-09

Initial Energy: 5.940e+02
Initia

100%|██████████| 8640/8640 [00:26<00:00, 327.55it/s]



LowRes_64x32 Simulation Complete!
  Loss: 0.186505
  → Trust region shrunk to 0.05

Status:
  Valid: 100/100
  Failed: 0
  Best loss: 0.127704 (iteration 59)
  Iterations w/o improvement: 11/15
  ✓ Progress saved

OPTIMIZATION COMPLETE

Total iterations: 100
  Valid: 100
  Failed: 0

Best loss: 0.127704
Best parameters:
  viscosity_scale: 2.601599e+00
  drag_scale: 1.077783e+00
  eddy_diffusivity: 4.651542e+04
  smagorinsky_coeff: 1.692916e-01
  energy_correction: -1.029489e-04
  enstrophy_correction: 7.289915e-09

✓ Saved: enhanced_gp_optimal_params.pkl
✓ Saved: enhanced_gp_optimal_config.txt
